In [1]:
import pandas as pd
import os
import csv
import numpy as np
from scipy import stats
from matplotlib import pyplot as plt
%matplotlib

Using matplotlib backend: Qt5Agg


In [ ]:
from scipy.ndimage.filters import median_filter

In [5]:
#en Experimentos está FLIR y en Experimentos2 están temp y tempcup y temp4cup
from Experimentos import Experimentos, labelLines

[Errno 2] No such file or directory: '.\\Data\\5cc50C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\1cc54C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\25cc54C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\10cc60C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\25cc60C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\1cc59C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\5cc63C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\25cc62C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\1cc62C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\10cc55C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\1cc24C - flir.csv'
[Errno 2] No such file or directory: '.\\Data\\10cc24C - flir.csv'


In [126]:
#Para revisar en que experimentos tengo archivo flir
#for i, exp in enumerate(Experimentos):
 #   print(i, (exp.flir.dtypes if exp.flir is not None else 0))

In [4]:
#Experimentos[14].flir.describe()

,FLIR: tiempo,FLIR: area,FLIR: rango 1,FLIR: rango 2,FLIR: rango 3,FLIR: rango 4,FLIR: rango 5,FLIR: rango 6,FLIR: rango 7,FLIR: rango 8,FLIR: rango 9,moving average,moving std
count,1557.000000,1557.000000,1557.000000,1557.000000,1557.000000,1557.000000,1557.000000,1557.000000,1557.000000,1557.000000,1557.000000,1557.000000,1557.000000
mean,785.181118,94162.901092,0.112485,0.120258,0.211461,0.182675,0.184660,0.087382,0.063736,0.033939,0.003378,94155.919717,469.647036
std,455.769516,20014.599567,0.085897,0.103205,0.164399,0.154823,0.228621,0.157622,0.146061,0.101394,0.016267,19989.444232,907.503220
min,0.000000,299.000000,0.015796,0.005306,0.005876,0.000030,0.000000,0.000000,0.000000,0.000000,0.000000,6333.500000,38.071816
25%,391.000000,95783.000000,0.032509,0.020066,0.038780,0.029736,0.000000,0.000000,0.000000,0.000000,0.000000,95764.000000,92.242586
50%,781.000000,101901.000000,0.085578,0.100824,0.189249,0.152571,0.043643,0.000000,0.000000,0.000000,0.000000,101891.500000,123.731405
75%,1180.000000,103825.000000,0.188446,0.195695,0.389261,0.331465,0.344386,0.103819,0.000000,0.000000,0.000000,103799.000000,220.558973
max,1578.000000,104805.000000,0.327759,0.354756,0.443221,0.450265,0.653564,0.575844,0.591025,0.527886,0.124881,104703.500000,5446.076358


In [3]:
numeropixeles = 1020100
rho_manjar = 1340
kilos_manjar = 2

In [6]:
#genero ESPESORES a partir de AREAS aproximando el flujo a un ELIPSOIDE REAL. 
for exp in Experimentos:
    if exp.flir is not None:
        try:
            exp.flir['area_m2'] = exp.flir['FLIR: area']/numeropixeles
            exp.flir['volumen_emitido'] = exp.param['caudal'][0]*exp.flir['FLIR: tiempo']
        except:
            continue
        volumen_total = kilos_manjar/rho_manjar
        exp.flir['volumen_emitido'][exp.flir['volumen_emitido'] > volumen_total] = volumen_total
        exp.flir['espesor'] = 3*exp.flir['volumen_emitido']/(2*exp.flir['area_m2'])

C:\Users\CCVV\Anaconda3b\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [33]:
#funcion que entrega los rangos de temperatura de cada uno de los exp
#exp.get_ranges()

In [ ]:
#FALTA: graficar tiempo en que prevalece cada rango de temperatura? y como relacionarlo a la velocidad

In [5]:
#achico los rangos

for exp in Experimentos:
    if exp.flir is not None:
        try:
            exp.flir['rango 1'] = exp.flir['FLIR: rango 1']+exp.flir['FLIR: rango 2']+exp.flir['FLIR: rango 3']
            exp.flir['rango 2'] = exp.flir['FLIR: rango 4']+exp.flir['FLIR: rango 5']+exp.flir['FLIR: rango 6']
            exp.flir['rango 3'] = exp.flir['FLIR: rango 7']+exp.flir['FLIR: rango 8']+exp.flir['FLIR: rango 9']

        except:
            continue

In [11]:
Experimentos[18].flir.describe()

,FLIR: tiempo,FLIR: area,FLIR: rango 1,FLIR: rango 2,FLIR: rango 3,FLIR: rango 4,FLIR: rango 5,FLIR: rango 6,FLIR: rango 7,FLIR: rango 8,FLIR: rango 9,moving average,moving std,area_m2,volumen_emitido,espesor,rango 1,rango 2,rango 3
count,1941.000000,1941.000000,1941.000000,1941.000000,1941.000000,1941.000000,1941.000000,1941.000000,1941.000000,1941.000000,1941.000000,1941.000000,1941.000000,1941.000000,1941.000000,1941.000000,1941.000000,1941.000000,1941.000000
mean,980.691396,98438.467285,0.133829,0.094392,0.221894,0.237164,0.149863,0.090145,0.050646,0.019551,0.002513,98418.919887,549.607886,0.096499,0.001458,0.026405,0.450116,0.477171,0.072710
std,571.134230,25921.205777,0.069117,0.068557,0.199774,0.219706,0.224876,0.182448,0.135069,0.069909,0.017171,25932.404918,818.010787,0.025410,0.000182,0.015269,0.332145,0.306911,0.191288
min,0.000000,266.000000,0.031947,0.005960,0.005136,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3864.000000,103.929581,0.000261,0.000000,0.000000,0.043740,0.000000,0.000000
25%,485.000000,95654.000000,0.069439,0.019864,0.045267,0.021062,0.000000,0.000000,0.000000,0.000000,0.000000,95592.000000,226.480649,0.093769,0.001493,0.019920,0.140505,0.211740,0.000000
50%,977.000000,108627.000000,0.131326,0.097245,0.153193,0.168842,0.000000,0.000000,0.000000,0.000000,0.000000,108637.500000,276.723004,0.106487,0.001493,0.021020,0.377826,0.509094,0.000000
75%,1475.000000,114647.000000,0.194750,0.157752,0.375867,0.445913,0.246267,0.054320,0.000000,0.000000,0.000000,114663.500000,400.504928,0.112388,0.001493,0.023856,0.728320,0.742178,0.000000
max,1974.000000,117211.000000,0.590226,0.209897,0.590474,0.642939,0.684389,0.740530,0.598064,0.413919,0.181936,116793.500000,5552.218874,0.114901,0.001493,0.090789,1.000000,0.941984,0.752497


In [90]:
listarangos = []

for exp in Experimentos:
    if exp.flir is not None:
        try:
            holder = {} #es un diccionario 
            holder['nombre'] = exp.param['NOMBRE'][0]#solo el valor de la primera fila
            holder['rango 1'] = exp.flir['rango 1']#toda la columna
            holder['rango 2'] = exp.flir['rango 2']#toda la columna
            holder['rango 3'] = exp.flir['rango 3']#toda la columna
            holder['avance'] = exp.dinamica['avance: largo total flujo']#toda la columna
            holder['tiempoavance'] = exp.dinamica['avance: tiempo']#toda la columna
            holder['tiempoflir'] = exp.flir['FLIR: tiempo']#toda la columna
            holder['caudal'] = exp.param['caudal'][0]#solo el valor de la primera fila
            holder['angulo'] = exp.param['angulo inclinacion'][0]
            holder['uso'] = exp.param['uso manjar'][0]

            listarangos.append(holder)
        except Exception as e:
            print(e)

In [125]:
#grafico de rangos por caudal

styles= ['red', 'sienna', 'darkgreen', 'purple', 'darkcyan', 'coral', 'navy', 
        'springgreen', 'magenta', 'y', 'palevioletred', 'blue']

fig = plt.figure()
axes = []


for x in range(c):
    axes.append(fig.add_subplot(2, 3, x + 1))

for i, (key, group) in enumerate(groupby(sorted(listarangos, key=lambda x: x['caudal']), key=lambda x: x['caudal'])):
    print(key, c)
    if key == 1e-06:
        c = 0     
        for exp in group:
            colors = styles[c]
            ax.set_title(exp['nombre'])

            axes[c].plot(exp['tiempo'], exp['rango 1'], color=colors)
            axes[c].plot(exp['tiempo'], exp['rango 2'], color=colors)
            axes[c].plot(exp['tiempo'], exp['rango 3'], color=colors)
            #axes[c].plot(exp['otrotiempo'], exp['distancia'], color=colors)
           
            c = c + 1
        break
    
fig.show()

1e-06 6


In [25]:
for exp in Experimentos:
    if exp.flir is not None:
        try:
            print(exp.flir['rango 1'].max())
            print(exp.flir['rango 2'].max())
            print(exp.flir['rango 3'].max())

        except:
            continue

1.0
0.747871812
0.619908466
0.565432179
0.846303847
0.699669967
1.000000001
0.93413261
0.90354621
1.000000001
0.934428167
0.906165947
0.999959626
0.934389829
0.898852245
0.999919647
0.970162376
0.859032918
1.000000001
0.951593036
0.832425786
0.999970252
0.928591873
0.848566546
0.60106383
0.939085894
0.85113972
1.000000001
0.94198386
0.752496543
0.662088922
0.901257787
0.812531838
0.689476333
0.944947648
0.855160069
1.000000001
0.93358557
0.903052395
1.000000001
0.956743889
0.887895111
1.000000038
0.795114777
0.465571764
1.000000001
0.933673238
0.796978281
0.820458661
0.770635781
0.72947053
1.000000044
0.933196364
0.771124557
1.000000001
0.907328555
0.807389334
1.000000001
0.943459649
0.835539401
0.725556153
0.870059399
0.746544606
0.901354672
0.832376863
0.699649738


In [7]:
#GENERA PATCHES DE COLORES SEGUN TEMPERATURA
import matplotlib.patches as mpatches

pat1 = mpatches.Patch(color='C1', label='temp < 40')
pat2 = mpatches.Patch(color='C2', label='temp < 45')
pat3 = mpatches.Patch(color='C3', label='temp < 50')
pat4 = mpatches.Patch(color='C4', label='temp < 55')
pat5 = mpatches.Patch(color='C5', label='temp < 60')
pat6 = mpatches.Patch(color='C6', label='temp < 65')
pat7 = mpatches.Patch(color='C7', label='temp < 70')

numerocolores = 7
colors = ['pat{}'.format(i+1) for i in range(numerocolores)]

print(colors)

['pat1', 'pat2', 'pat3', 'pat4', 'pat5', 'pat6', 'pat7']


In [18]:
velos = pd.read_excel('Velocidades y punto.xlsx')

In [19]:
for exp in Experimentos:
    exp.param['tiempo fin eyeccion'] = exp.param['volumen total teorico']/exp.param['caudal']

In [35]:
#LISTA para ESPESOR vs TIEMPO
listaporcaudal = []

for exp in Experimentos:
    if exp.flir is not None:
        try:
            holder = {} #es un diccionario 
            holder['nombre'] = exp.param['NOMBRE'][0]#solo el valor de la primera fila
            holder['espesor'] = exp.flir['espesor']#toda la columna
            holder['tiempo'] = exp.flir['FLIR: tiempo']#toda la columna
            holder['caudal'] = exp.param['caudal'][0]#solo el valor de la primera fila
            holder['angulo'] = exp.param['angulo inclinacion'][0]
            holder['uso'] = exp.param['uso manjar'][0]
            holder['teyec'] = exp.param['tiempo fin eyeccion']
            holder['espesorfinal'] = exp.perfil['perfil: espesor'].max()



            listaporcaudal.append(holder)
        except Exception as e:
            print(e)

In [38]:
#for exp in Experimentos:
#    print(exp.flir['espesor'] if exp.flir is not None else None)

In [9]:
#crear GRUPOS segun CAUDAL
from itertools import groupby
count = 0

#key es la lista donde estan los diccionarios de exp y DF
#group es que se agrupa segun el valor de 'caudal' en este caso

for key, group in groupby(sorted(listaporcaudal, key=lambda x: x['caudal']), key=lambda x: x['caudal']):
    print(key)
    count += 1
    #for exp in group:
        #muestra el nombre del grupo y los elementos que tiene
        #print(exp['nombre']) 

1e-06
5e-06
1e-05
1.7e-05
2.5e-05


In [39]:
#ESPESOR vs TIEMPO agrupados por caudales
fig = plt.figure()
axes = [fig.add_subplot(3,2,i+1) for i in range(count)]
counter = 0

for key, group in groupby(sorted(listaporcaudal, key=lambda x: x['caudal']), key=lambda x: x['caudal']):
    print(key)
    c = 0
    ax = axes[counter]
    #ax.set_xlim(minimoejelargo, maximoejelargo)
    #ax.set_ylim(0, maximoejeespesor)
    ax.set_title('{}cc'.format(key*1000000))
    counter += 1
    #linea1= ax.plot([],[], label='15°', linestyle='--', color='black')
    #linea2= ax.plot([],[], label='10°', color='black')
    
    for exp in group:
        #print("C{}".format(exp["rango temperatura"]))
        #if exp['uso'] == 1:
            c = c + 1
            ax.plot(exp['tiempo'], exp['espesor'], label = '')
            ax.set_xlabel('Tiempo [s]')
            ax.set_ylabel('Espesor teórico [m]')
            ax.axhline(exp['espesorfinal'])

        
legend1 = plt.legend(handles=[pat1, pat2, pat3, pat4, pat5, pat6, pat7], bbox_to_anchor=(1.8, 0.5), 
                     loc=5, title='Rango temperatura inicial del flujo')
plt.legend(bbox_to_anchor=(1.8, 0.5), loc=6)
plt.gca().add_artist(legend1)
fig.show()

1e-06
5e-06
1e-05
1.7e-05
2.5e-05


C:\Users\CCVV\Anaconda3b\lib\site-packages\matplotlib\axes\_axes.py:545: UserWarning: No labelled objects found. Use label='...' kwarg on individual plots.
  warnings.warn("No labelled objects found. "


In [16]:
#LISTA para AREA vs TIEMPO
otralistaporcaudal = []

for exp in Experimentos:
    if exp.flir is not None:
        try:
            holder = {} #es un diccionario 
            holder['nombre'] = exp.param['NOMBRE'][0]#solo el valor de la primera fila
            holder['area'] = exp.flir['area_m2']#toda la columna
            holder['tiempo'] = exp.flir['FLIR: tiempo']#toda la columna
            holder['caudal'] = exp.param['caudal'][0]#solo el valor de la primera fila
            holder['angulo'] = exp.param['angulo inclinacion'][0]
            holder['uso'] = exp.param['uso manjar'][0]

            otralistaporcaudal.append(holder)
        except:
            continue

In [23]:
#AREA vs TIEMPO agrupados por caudales
fig = plt.figure()
axes = [fig.add_subplot(3,2,i+1) for i in range(count)]
counter = 0

for key, group in groupby(sorted(otralistaporcaudal, key=lambda x: x['caudal']), key=lambda x: x['caudal']):
    print(key)
    ax = axes[counter]
    #ax.set_xlim(minimoejelargo, maximoejelargo)
    #ax.set_ylim(0, maximoejeespesor)
    ax.set_title('{}cc'.format(key*1000000))
    counter += 1
    
    for exp in group:
        ax.plot(exp['tiempo'], exp['area'])
        
legend1 = plt.legend(handles=[pat1, pat2, pat3, pat4, pat5, pat6, pat7], bbox_to_anchor=(1.8, 0.5), loc=5)
#plt.legend(bbox_to_anchor=(1.8, 0.5), loc=6)
plt.gca().add_artist(legend1)
fig.show()

1e-06
5e-06
1e-05
1.7e-05
2.5e-05


In [17]:
for k, g in groupby(sorted(otralistaporcaudal, key=lambda x: x['caudal']), key=lambda x: x['caudal']):
    print(k, g)

1e-06 <itertools._grouper object at 0x000001EC2E777080>
5e-06 <itertools._grouper object at 0x000001EC2DEA6BA8>
1e-05 <itertools._grouper object at 0x000001EC2E774FD0>
1.7e-05 <itertools._grouper object at 0x000001EC2E774BA8>
2.5e-05 <itertools._grouper object at 0x000001EC2E774D68>
